In [ ]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

In [ ]:
# Cargar los datos desde los archivos proporcionados
costos_df = pd.read_excel('Costos_Subprocesos.xlsx')
indicadores_minas_df = pd.read_excel('Indicadores_Minas.xlsx')
indicadores_carguio_df = pd.read_excel('Indicadores_Carguio.xlsx')
indicadores_perforacion_df = pd.read_excel('Indicadores_Perforación.xlsx')
indicadores_transporte_df = pd.read_excel('Indicadores_Transporte.xlsx')

# Convertir las fechas a formato datetime en costos
costos_df['Subproceso_Costo'] = pd.to_datetime(costos_df['Subproceso_Costo'])

# Convertir las fechas a formato datetime en los DataFrames de indicadores
for df in [indicadores_minas_df, indicadores_carguio_df, indicadores_perforacion_df, indicadores_transporte_df]:
    df['Fecha'] = pd.to_datetime(df['Fecha'])

# Unir todos los DataFrames de indicadores en uno solo
indicadores_df = indicadores_minas_df.copy()
indicadores_df = pd.merge(indicadores_df, indicadores_carguio_df, on='Fecha', how='inner', suffixes=('_minas', '_carguio'))
indicadores_df = pd.merge(indicadores_df, indicadores_perforacion_df, on='Fecha', how='inner', suffixes=('', '_perforacion'))
indicadores_df = pd.merge(indicadores_df, indicadores_transporte_df, on='Fecha', how='inner', suffixes=('', '_transporte'))

# Unir el DataFrame de costos con el DataFrame combinado de indicadores
df_unificado = pd.merge(costos_df, indicadores_df, left_on='Subproceso_Costo', right_on='Fecha', how='inner')
df_unificado = df_unificado.drop(columns=['Fecha'])  # Eliminar la columna duplicada de Fecha

# Calcular la correlación entre todos los subprocesos y los indicadores usando Spearman
subprocesos_cols = costos_df.columns[1:]  # Excluyendo la columna de fecha
indicadores_cols = indicadores_df.columns[1:]  # Excluyendo la columna de fecha
correlation_matrix = df_unificado.corr(method='spearman')
correlation_filtered = correlation_matrix.loc[subprocesos_cols, indicadores_cols]
top_correlations = correlation_filtered.apply(lambda x: x.nlargest(3), axis=1)

In [52]:
# Definir los modelos a evaluar
modelos = {
    'Linear Regression': LinearRegression(),
    'Polynomial Regression (Degree 2)': PolynomialFeatures(degree=2),
    'Polynomial Regression (Degree 3)': PolynomialFeatures(degree=3),
    'Polynomial Regression (Degree 4)': PolynomialFeatures(degree=4),
    'ElasticNet': ElasticNet(alpha=1.0, l1_ratio=0.5, random_state=42),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
    'Gradient Boosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    'XGBoost': XGBRegressor(n_estimators=100, random_state=42),
    'LightGBM': LGBMRegressor(n_estimators=100, random_state=42),
    'SVM': SVR(),
    'MLP': MLPRegressor(hidden_layer_sizes=(64, 32), max_iter=500, random_state=42)
}

# Crear un diccionario para almacenar los resultados de cada modelo
resultados_modelos = {}
indicadores_utilizados = {}

# Escalar los datos
scaler = StandardScaler()

# Crear imputador para reemplazar NaNs con la media
imputer = SimpleImputer(strategy='mean')

X_train_scaled = scaler.fit_transform(imputer.fit_transform(train_data[indicadores_cols]))
X_test_scaled = scaler.transform(imputer.transform(test_data[indicadores_cols]))

# Realizar predicciones para cada modelo
for nombre_modelo, modelo in modelos.items():
    resultados = {}
    for subproceso in subprocesos_cols:
        # Seleccionar los indicadores más correlacionados (en este caso, los 3 más altos)
        indicadores_seleccionados = top_correlations.loc[subproceso].dropna().index.tolist()
        
        if len(indicadores_seleccionados) > 0:
            # Guardar los indicadores seleccionados
            indicadores_utilizados[subproceso] = indicadores_seleccionados
            
            # Utilizar todos los indicadores seleccionados para el entrenamiento
            X_train = train_data[indicadores_seleccionados]
            y_train = train_data[subproceso]
            X_test = test_data[indicadores_seleccionados]
            
            # Imputar valores faltantes en los conjuntos de datos
            X_train = imputer.fit_transform(X_train)
            X_test = imputer.transform(X_test)
            
            # Verificar si y_train tiene valores constantes
            if y_train.nunique() == 1:
                print(f"Advertencia: El subproceso {subproceso} tiene un objetivo constante en el conjunto de entrenamiento.")
                continue  # Omitir este subproceso
            
            # Aplicar el modelo correspondiente
            if 'Polynomial' in nombre_modelo:
                poly = modelo
                X_train_poly = poly.fit_transform(X_train)
                X_test_poly = poly.transform(X_test)
                lin_reg = LinearRegression()
                lin_reg.fit(X_train_poly, y_train)
                y_pred_2022 = lin_reg.predict(X_test_poly)
            else:
                modelo.fit(X_train, y_train)
                y_pred_2022 = modelo.predict(X_test)

            costo_real_2022 = test_data[subproceso].values
            
            # Calcular las diferencias y diferencias porcentuales
            diferencias = y_pred_2022 - costo_real_2022
            diferencias_pct = [(dif / real) * 100 if real != 0 else np.inf for dif, real in zip(diferencias, costo_real_2022)]
            
            # Guardar resultados en el diccionario
            for mes, pred, real, dif, dif_pct in zip(pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"), y_pred_2022, costo_real_2022, diferencias, diferencias_pct):
                resultados.setdefault(subproceso, {}).update({
                    f'Predicción {mes}': pred,
                    f'Costo Real {mes}': real,
                    f'Diferencia {mes}': dif,
                    f'Diferencia % {mes}': dif_pct
                })
    
    # Convertir los resultados a un DataFrame
    resultados_df = pd.DataFrame(resultados).T
    resultados_modelos[nombre_modelo] = resultados_df

# DataFrames para los mejores modelos y diferencias
mejores_modelos_df = pd.DataFrame(index=subprocesos_cols, columns=pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"))
diferencias_df = pd.DataFrame(index=subprocesos_cols, columns=pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"))

# Comparar los resultados entre modelos para cada subproceso y mes
for subproceso in subprocesos_cols:
    for mes in pd.date_range('2022-01-01', '2022-12-01', freq='MS').strftime("%B-%Y"):
        mejor_modelo = None
        menor_diferencia = np.inf
        
        for nombre_modelo, df in resultados_modelos.items():
            diferencia_col = f'Diferencia % {mes}'
            if diferencia_col in df.columns and subproceso in df.index and df.loc[subproceso, diferencia_col] != np.inf:
                diferencia = abs(df.loc[subproceso, diferencia_col])
                if diferencia < menor_diferencia:
                    menor_diferencia = diferencia
                    mejor_modelo = nombre_modelo
        
        # Guardar el mejor modelo y su diferencia
        mejores_modelos_df.loc[subproceso, mes] = mejor_modelo
        diferencias_df.loc[subproceso, mes] = menor_diferencia

# Crear un DataFrame de resumen
resumen_df = pd.DataFrame(index=subprocesos_cols, columns=['Modelo Recomendado', 'Diferencia Promedio'])

for subproceso in subprocesos_cols:
    modelos_mes = mejores_modelos_df.loc[subproceso].value_counts()
    
    if not modelos_mes.empty:
        modelo_recomendado = modelos_mes.idxmax()  # El modelo que más se repite
    else:
        modelo_recomendado = "N/A"  # Si no hay datos válidos, asigna "N/A" o algún valor predeterminado
    
    promedio_diferencia = diferencias_df.loc[subproceso].replace([np.inf, 0], np.nan).mean()  # Promedio ajustado
    resumen_df.loc[subproceso] = [modelo_recomendado, promedio_diferencia]

# Calcular la precisión total como la media de todas las diferencias
precision_total = diferencias_df.replace([np.inf, 0], np.nan).mean().mean()

# Exportar los resultados a un archivo Excel con las hojas adicionales
output_file_path = 'Resultados_Final_Modelos2.xlsx'

with pd.ExcelWriter(output_file_path) as writer:
    # Guardar el DataFrame con los mejores modelos para cada subproceso y mes
    mejores_modelos_df.to_excel(writer, sheet_name='Mejores Modelos')
    
    # Guardar el DataFrame con las diferencias porcentuales
    diferencias_df.to_excel(writer, sheet_name='Diferencias %')
    
    # Guardar el resumen con el modelo recomendado y la diferencia promedio ajustada
    resumen_df.to_excel(writer, sheet_name='Resumen')
    
    # Guardar los indicadores utilizados para cada subproceso
    indicadores_df = pd.DataFrame.from_dict(indicadores_utilizados, orient='index')
    indicadores_df.to_excel(writer, sheet_name='Indicadores Utilizados')

# Imprimir la precisión total
print(f"Precisión total (promedio ajustado de diferencias): {precision_total:.2f}%")

Advertencia: El subproceso SERVICIOS TERCEROS LLANO tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso TRONADURA TESORO NOR ESTE tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso SERVICIOS TERCEROS LLANO tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso TRONADURA TESORO NOR ESTE tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso SERVICIOS TERCEROS LLANO tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso TRONADURA TESORO NOR ESTE tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso SERVICIOS TERCEROS LLANO tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso TRONADURA TESORO NOR ESTE tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso SERVICIOS TERCEROS LLANO tiene un objetivo constante en el conjun

C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 9.409e+09, tolerance: 1.370e+07
  model = cd_fast.enet_coordinate_descent(
C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.706e+13, tolerance: 5.911e+09
  model = cd_fast.enet_coordinate_descent(
C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:678: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.736e+

Advertencia: El subproceso TRONADURA TESORO NOR ESTE tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso SERVICIOS TERCEROS LLANO tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso TRONADURA TESORO NOR ESTE tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso SERVICIOS TERCEROS LLANO tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso TRONADURA TESORO NOR ESTE tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso SERVICIOS TERCEROS LLANO tiene un objetivo constante en el conjunto de entrenamiento.
Advertencia: El subproceso TRONADURA TESORO NOR ESTE tiene un objetivo constante en el conjunto de entrenamiento.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000031 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `f

C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\n

Advertencia: El subproceso SERVICIOS TERCEROS LLANO tiene un objetivo constante en el conjunto de entrenamiento.


C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\l.vilches\anaconda3\Lib\site-packages\sklearn\n

Advertencia: El subproceso TRONADURA TESORO NOR ESTE tiene un objetivo constante en el conjunto de entrenamiento.
Precisión total (promedio ajustado de diferencias): 1789274028.63%


C:\Users\l.vilches\AppData\Local\Temp\ipykernel_22880\4098151448.py:163: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  promedio_diferencia = diferencias_df.loc[subproceso].replace([np.inf, 0], np.nan).mean()  # Promedio ajustado
C:\Users\l.vilches\AppData\Local\Temp\ipykernel_22880\4098151448.py:163: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  promedio_diferencia = diferencias_df.loc[subproceso].replace([np.inf, 0], np.nan).mean()  # Promedio ajustado
C:\Users\l.vilches\AppData\Local\Temp\ipykernel_22880\4098151448.py:163: Futur